In [17]:
from pathlib import Path

In [87]:
def parser(root:str)->dict:
    '''
    Parser thar explore root dir to find all .txt files and read them into returning dictionary
    1. root: path (str format) of root dir to be explored
    2. catalogue: dictionary with all information from .txt files
    '''
    root_path = Path(root)
    counter = -1
    catalogue = {} # returning dict with HL7 data
    
    for txt_file_path in root_path.rglob('*.txt'): # find all .txt files with recursion
        with txt_file_path.open('r') as file:
            temp_dict = {}
            counter += 1
            
            for line in file:
                line = line.strip()
                filtered_line = list(filter(None, line.split('|')))
                head = filtered_line.pop(0) 
                
                if head in temp_dict: # if segment already exist:
                    if isinstance(temp_dict[head], list): # check is it list or not
                        temp_dict[head].append(filtered_line) # if value of segment if list, just add data to end of list
                    else:
                        temp_dict[head] = [temp_dict[head], filtered_line] # if not, convert it into list format and add data to the end 
                else:
                    temp_dict.setdefault(head, None) # if segment is abcent, create the key
                    temp_dict[head] = filtered_line # and add the data

                
            catalogue.setdefault(counter, None)
            catalogue[counter] = temp_dict
            temp_dict = {}
    return catalogue

In [88]:
root_dir = r'C:\Users\neuropromotion\Desktop\ML in healthcare\Clinical_HL7_Samples'
data = parser(root_dir)

In [94]:
data[55]

{'MSH': ['^~\\&',
  'WstrnRgnlMedCntr^1231231235^NPI',
  '201207171700',
  'ADT^A04^ADT_A01',
  'NIST-SS-002.11',
  'P',
  '2.5.1',
  'PH_SS-NoAck^SS Sender^2.16.840.1.114222.4.10.3^ISO'],
 'EVN': ['201207171700', 'WstrnRgnlMedCntr^1231231235^NPI'],
 'PID': ['1', '222^^^^MR', '^^^^^^~^^^^^^U', 'F', '2106-3^^CDCREC'],
 'PV1': ['1', '20120709_0064^^^^VN', '201207171700'],
 'OBX': ['1',
  'CWE',
  'SS003^^PHINQUESTION',
  '261QE0002X^Emergency Care^NUCC',
  'F',
  ['2', 'NM', '21612-7^^LN', 'UNK^^NULLFL', 'F'],
  ['3',
   'CWE',
   '8661-1^^LN',
   'E8126^Other motor vehicle traffic accident involving collision with motor vehicle injuring pedal cyclist^I9CDX',
   'F']]}

In [79]:
data[0]['PID'] 

5

In [97]:
# find youngest male patient

list_of_males_bd = []
for ctr in list(data.keys()):
    if 'M' in data[ctr]['PID']:
        ind = data[ctr]['PID'].index('M') - 1 # index of birthday data (tipically strictly before SEX)
        bd = data[ctr]['PID'][ind]
        if bd.isdigit(): # not all records have birthday data
            list_of_males_bd.append(bd)

In [98]:
from datetime import datetime

def convert_data(data:str):
    return datetime.strptime(data, "%Y%m%d").date()

males_bd = [convert_data(x) for x in list_of_males_bd]

In [101]:
# youngest patient
max(males_bd)

datetime.date(2010, 9, 7)